# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

<img src='topic.png'>

# About me

<a href="http://arudnitskiy.ru/">Read here</a>

<img src='about_me.png'>

<img src='structure.png'>

<img src='lesson_topic.png'>

-----

# 1. Введение и примеры задач

<img src='recsys_everywhere.png'>


----

<img src='skills.png'>

---

##  Постановка задачи

*Задача рекомендаций* - предложить пользователю товары, которые ему "понравятся"

Компоненты:
    - Пользователи (users)
    - Товары (items)
    - Взаимодействия (interactions) - по ним понимаем, что товар "понравился"

---

## Пользователи VS Товары

<img src='ex_user_items.png'>


**Кейс**: определить в каждой ситуации, что является пользователем, а что - товаром
    
    1. На сайте Кинопоиска порекомендовать интересные фильмы
    2. На сайте hh.ru подобрать лучшие вакансии для соискателя вакансий
    3. На сайте hh.ru найти похожие вакансии на ту, которую сейчас просматривает пользователь ("вам также может понравиться")
    4. На сайте hh.ru подобрать лучших кандидатов для работодателя
    
*Пример ответа в чате:*
    
    1. Пользователь (П) - фильм, Товар (T) - фильм
    
    2. Т - работодатель, П - соискатель вакансий

---

## Взаимодействия --> "понравился товар"

<img src='ex_interactions.png'>

---

## В чем отличие от классификации?

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
             ['Иван', 'Хлеб Белый', 0],
             ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Василий,Йогурт Epica,1


Постойте, ведь можно добавить фичи на user (средний чек, кол-во покупок в категориях и т.д), 
на item (цена, кол-во продаж в неделю и т.l), и решать задачу классификации. В чем отличие RecSys от классификации?

 - Много предсказаний для 1 user (extreme classification)
 - Гораздо больший объем данных: 100K users, 10K items --> 1B предсказаний
 - Большинство товаров user никогда не видел --> Не взаимодействовал --> 0 не значит "не понравился"
 - Нет явного таргета. Не понятно, что значит "(не) понравился"
 - Feedback loop
 - Всегда важен порядок рекомендаций

In [3]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
              ['Иван', 'Хлеб Белый', 0],
              ['Иван', 'Йогурт Epica', "?"],
              ['Василий', 'Хлеб Бородинский', "?"],
              ['Василий', 'Хлеб Белый', "?"],
              ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Иван,Йогурт Epica,?
3,Василий,Хлеб Бородинский,?
4,Василий,Хлеб Белый,?
5,Василий,Йогурт Epica,1


----

## Основные типы задач:
    
**Рекомендация топ-К товаров**: Дан список товаров. Рекомендовать пользователю K товаров, которые ему понравятся
    - e-mail рассылки (М.Видео, Hoff, Пятерочка)
    - push-уведомления (Delivery Club, HeadHunter, Виктория)
    - Рекомендации в отдельном окне на сайте (vk, okko)

**Ранжирование товаров**: Дан список товаров. Нужно его отранжировать в порядке убывания интереса для пользователя
    - Ранжирование каталога товаров (Wildberries, Lamoda)
    - Ранжирование ленты (vk, Я.Дзен)
    - Ранжирование поисковой выдачи (Яндекс, Гугл)
    - Ранжирование "каруселей" товаров (Delivery Club, Я.Еда)
    
**Поиск похожих товаров**: Дан 1 товар. Нужно найти наиболее похожие на него товары
    - "Вам также может понравиться" (Ozon, Циан)
    - Похожим пользователям понравилось (vk)
    - Вы можете быть знакомы (vk, facebook)
    
**Рекомендация дополнительного товара**. Дан 1 товар. Найти товары, которые покупают вместе с этим товаром
    - С этим товаром часто покупают (Я.Маркет, Ozon, М.Видео)
    
   
---
На 7 вебинаре мы разберем рекомендательные системы из HeadHunter

---

# 2. ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [4]:
# список товаров, которые мы порекомендуем пользователю в заданном порядке
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров

# юзер 1 купил данные товары
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [5]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

In [6]:
# без функции hit_rate:

def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

In [7]:
# с функцией hit_rate:

def hit_rate_at_k_2(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [8]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [9]:
hit_rate(recommended_list, bought_list)

1

In [10]:
hit_rate_at_k(recommended_list, bought_list)

1

In [11]:
hit_rate_at_k_2(recommended_list, bought_list)

1

In [12]:
recommended_list = [156, 1134, 27, 1543, 3345, 533, 11, 43, 143] #id товаров
bought_list = [521, 32, 143, 991]

In [13]:
hit_rate_at_k(recommended_list, bought_list)

0

In [14]:
hit_rate_at_k_2(recommended_list, bought_list)

0

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [15]:
recommended_list = [2, 3, 8, 6, 9, 2, 5, 10] #id товаров
bought_list = [1, 5, 6]

In [16]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)

In [17]:
precision(recommended_list, bought_list)

0.25

In [18]:
# проверка:

(0 + 0 + 0 + 1 + 0 + 0 + 1 + 0) / (1 + 1 + 1 + 1 + 1 + 1 + 1 + 1)

0.25

In [19]:
# без функции precision:

def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)

In [20]:
# с функцией precision:

def precision_at_k_2(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [21]:
precision_at_k(recommended_list, bought_list, k=3)

0.0

In [22]:
precision_at_k_2(recommended_list, bought_list, k=3)

0.0

In [23]:
# проверка:

(0 + 0 + 0) / (1 + 1 + 1)

0.0

In [24]:
precision_at_k(recommended_list, bought_list, k=5)

0.2

In [25]:
precision_at_k_2(recommended_list, bought_list, k=5)

0.2

In [26]:
# проверка:

(0 + 0 + 0 + 1 + 0) / (1 + 1 + 1 + 1 + 1)

0.2

In [27]:
recommended_list = [2, 5, 3, 8, 6, 9, 2, 10] #id товаров
bought_list = [1, 5, 6]

In [28]:
precision(recommended_list, bought_list)

0.25

In [29]:
# проверка:

(0 + 1 + 0 + 0 + 1) / (1 + 1 + 1 + 1 + 1 + 1 + 1 + 1)

0.25

In [30]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [31]:
precision_at_k_2(recommended_list, bought_list, k=5)

0.4

In [32]:
# проверка:

(0 + 1 + 0 + 0 + 1) / (1 + 1 + 1 + 1 + 1)

0.4

In [33]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [34]:
prices = [23, 2233, 534, 56, 1]
money_precision_at_k(recommended_list, bought_list, prices)

0.7846856340007025

In [35]:
# проверка:

(0 * 23 + 1 * 2233 + 0 * 534 + 0 * 56 + 1 * 1) / (1 * 23 + 1 * 2233 + 1 * 534 + 1 * 56 + 1 * 1)

0.7846856340007025

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [36]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)


'''
или:

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)
'''


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return recall

In [37]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices = [23, 2233, 534, 56, 1]
prices_bought = [300, 100, 23, 4]

In [38]:
recall(recommended_list, bought_list)

0.25

In [39]:
# проверка:

(1 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0) / 4

0.25

In [40]:
recall_at_k(recommended_list, bought_list, k=5)

0.25

In [41]:
# проверка:

(1 + 0 + 0 + 0 + 0) / 4

0.25

In [42]:
money_recall_at_k(recommended_list, bought_list, prices, prices_bought)

0.053864168618266976

In [43]:
# проверка:

(1 * 23 + 0 * 2233 + 0 * 534 + 0 * 56 + 0 * 1) / (300 + 100 + 23 + 4)

0.053864168618266976

In [44]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [143, 156]
prices = [23, 2233, 534, 56, 1]
prices_bought = [23, 2233]

In [45]:
recall(recommended_list, bought_list)

1.0

In [46]:
# проверка:

(1 + 1 + 0 + 0 + 0 + 0 + 0 + 0 + 0) / 2

1.0

In [47]:
recall_at_k(recommended_list, bought_list, k=5)

1.0

In [48]:
# проверка:

(1 + 1 + 0 + 0 + 0) / 2

1.0

In [49]:
money_recall_at_k(recommended_list, bought_list, prices, prices_bought)

1.0

In [50]:
# проверка:

(1 * 23 + 1 * 2233 + 0 * 534 + 0 * 56 + 0 * 1) / (23 + 2233)

1.0

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантных среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [51]:
recommended_list = [1, 21, 22, 4, 234, 232432, 234234, 666] #id товаров
bought_list = [1, 221, 3, 4]

In [52]:
precision(recommended_list, bought_list)

0.25

In [53]:
# проверка:

(1 + 0 + 0 + 1 + 0) / (1 + 1 + 1 + 1 + 1 + 1 + 1 + 1)

0.25

In [54]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    
    '''
    или без использования функции precision_at_k:
        
    sum_ = sum([(np.isin(bought_list, recommended_list[:index_relevant + 1])).sum() / 
                len(recommended_list[:index_relevant + 1]) for index_relevant in relevant_indexes])
    '''
    
    return sum_ / amount_relevant

In [55]:
ap_k(recommended_list, bought_list, k=5)

0.75

In [56]:
# проверка:

1 / 2 * ((1 / 1 + (1 + 0 + 0 + 1) / (1 + 1 + 1 + 1)))

0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [57]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], # юзер 1
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345], # юзер 2
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11] # юзер 3
                    ]

bought_list_3_users = [[521, 32, 143], # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]] # юзер 3

In [58]:
# с использованием функции ap_k:

def map_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list, dtype=object)
    recommended_list = np.array(recommended_list, dtype=object)[:k]
    
    assert len(recommended_list) == len(bought_list)
    
    sum_apk = sum([ap_k(recommended_list[i], bought_list[i], k) for i in range(len(bought_list))])
    amount_users = len(bought_list)
    
    result = sum_apk / amount_users
    
    return result

In [59]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

In [60]:
# без использования функции ap_k:

def map_k_(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list, dtype=object)
    recommended_list = np.array(recommended_list, dtype=object)[:k]
    
    assert len(recommended_list) == len(bought_list)
    
    apk = []
    
    for i in range(len(bought_list)):
        relevant_indexes = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
        
        if len(relevant_indexes) == 0:
            break
    
        amount_relevant = len(relevant_indexes)
        sum_ = sum([precision_at_k(recommended_list[i], bought_list[i], k=index_relevant + 1) for index_relevant in relevant_indexes])
        
        '''
        или без использования функции precision_at_k:
        
        sum_ = sum([(np.isin(bought_list[i], recommended_list[i][:index_relevant + 1])).sum() / 
                    len(recommended_list[i][:index_relevant + 1]) for index_relevant in relevant_indexes])
        '''
        
        apk.append(sum_ / amount_relevant)
    
    sum_apk = sum(apk)
    amount_users = len(bought_list_3_users)
    
    result = sum_apk / amount_users
    
    return result

In [61]:
map_k_(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

In [62]:
# проверка:

1 / 3 * ((1 / 1 * (1 / 1)) + 0 + 0)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [63]:
recommended_list = [156, 143, 1134, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

In [64]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list, dtype=object)
    recommended_list = np.array(recommended_list, dtype=object)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    dcg = 1 / len(recommended_list) * (sum([j / (i + 1) if i + 1 <= 2 else j / np.log2(i + 1) for i, j in enumerate(flags)]))
    ideal_dcg = 1 / len(recommended_list) * (sum([1 / (i + 1) if i + 1 <= 2 else 1 / np.log2(i + 1) for i in range(len(flags))]))
    ndcg = dcg / ideal_dcg
    
    return ndcg

In [65]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.16331296355715133

In [66]:
# проверка:

In [67]:
dcg = 1 / 5 * (0 / 1 + 1 / 2 + 0 / np.log2(3) + 0 / np.log2(4) + 0 / np.log2(5))
dcg

0.1

In [68]:
ideal_dcg = 1 / 5 * (1 / 1 + 1 / 2 + 1 / np.log2(3) + 1 / np.log2(4) + 1 / np.log2(5))
ideal_dcg

0.6123212623289702

In [69]:
ndcg = dcg / ideal_dcg
ndcg

0.16331296355715133

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [70]:
recommended_list = [156, 143, 1134, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], # юзер 1
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345], # юзер 2
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11] # юзер 3
                    ]

bought_list_3_users = [[521, 32, 143], # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]] # юзер 3

In [71]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list, dtype=object)
    recommended_list = np.array(recommended_list, dtype=object)[:k]

    if True in [isinstance(ii, list) for ii in bought_list]:
        assert len(recommended_list) == len(bought_list)
        mrr = []
        for i in range(len(bought_list)):
            rank = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
            if len(rank) == 0:
                break
            rank = rank[0] + 1
            rr = 1 / rank
            mrr.append(rr)
        return sum([1 / r for r in mrr]) / len(bought_list) if len(mrr) > 0 else 0
    else:
        rank = np.nonzero(np.isin(recommended_list, bought_list))[0]
        if len(rank) == 0:
            return 0
        else:
            rank = rank[0] + 1
            return 1 / rank

In [72]:
reciprocal_rank(recommended_list, bought_list, k=5)

0.5

In [73]:
# проверка:

1 / 2

0.5

In [74]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

In [75]:
# проверка:

1 / 3 * (1 / 1 + 0 + 0)

0.3333333333333333

### Другие примеры метрик для оценки рекомендаций/ранжирования

Cumulative gain - как discounted cumulative gain, только проще, без учёта порядка элементов (не нормализована и не учитывает позицию релевантных элементов).

Метрики на основе ранговой корреляции: ранговый коэффициент корреляции Кендэлла, ранговый коэффициент корреляции Спирмена.

Метрики на основе каскадной модели поведения: expected reciprocal rank (ERR), PFound

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возможно в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе

# Полезные ссылки

Статьи: https://docs.google.com/document/d/16L1u5zuQyT5rSAQNk2Q7Zk5KzqlPqWTpYa2cPK3QU0Y/edit?usp=sharing

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**